# 제품 이상여부 판별 프로젝트

## 1. 데이터 불러오기

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import xgboost as xgb
import shap
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA


pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

ROOT_DIR = "data"
RANDOM_STATE = 110

In [2]:
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))

drop_cols = []
for column in train_data.columns:
    if (train_data[column].notnull().sum() // 2) < train_data[column].isnull().sum():
        drop_cols.append(column)
train_data = train_data.drop(drop_cols, axis=1)

### OK Nan 행 이동

In [3]:
data_dam_columns = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam','HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam','HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
                       'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam','HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam','HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',
                       'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam','HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam','HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam',
                       'HEAD Standby Position X Collect Result_Dam','HEAD Standby Position Y Collect Result_Dam','HEAD Standby Position Z Collect Result_Dam','Head Clean Position X Collect Result_Dam',
                       'Head Clean Position Y Collect Result_Dam','Head Clean Position Z Collect Result_Dam','Head Purge Position X Collect Result_Dam','Head Purge Position Y Collect Result_Dam',
                       'Head Purge Position Z Collect Result_Dam','Head Zero Position X Collect Result_Dam','Head Zero Position Y Collect Result_Dam','Head Zero Position Z Collect Result_Dam',
                       'Machine Tact time Collect Result_Dam','PalletID Collect Result_Dam','Production Qty Collect Result_Dam','Receip No Collect Result_Dam','Stage1 Circle1 Distance Speed Collect Result_Dam',
                       'Stage1 Circle2 Distance Speed Collect Result_Dam','Stage1 Circle3 Distance Speed Collect Result_Dam','Stage1 Circle4 Distance Speed Collect Result_Dam',
                       'Stage1 Line1 Distance Speed Collect Result_Dam','Stage1 Line2 Distance Speed Collect Result_Dam','Stage1 Line3 Distance Speed Collect Result_Dam',
                       'Stage1 Line4 Distance Speed Collect Result_Dam','Stage2 Circle1 Distance Speed Collect Result_Dam','Stage2 Circle2 Distance Speed Collect Result_Dam',
                       'Stage2 Circle3 Distance Speed Collect Result_Dam','Stage2 Circle4 Distance Speed Collect Result_Dam','Stage2 Line1 Distance Speed Collect Result_Dam',
                       'Stage2 Line2 Distance Speed Collect Result_Dam','Stage2 Line3 Distance Speed Collect Result_Dam','Stage2 Line4 Distance Speed Collect Result_Dam',
                       'Stage3 Circle1 Distance Speed Collect Result_Dam','Stage3 Circle2 Distance Speed Collect Result_Dam','Stage3 Circle3 Distance Speed Collect Result_Dam',
                       'Stage3 Circle4 Distance Speed Collect Result_Dam','Stage3 Line1 Distance Speed Collect Result_Dam','Stage3 Line2 Distance Speed Collect Result_Dam',
                       'Stage3 Line3 Distance Speed Collect Result_Dam','Stage3 Line4 Distance Speed Collect Result_Dam','THICKNESS 1 Collect Result_Dam','THICKNESS 2 Collect Result_Dam',
                       'THICKNESS 3 Collect Result_Dam','WorkMode Collect Result_Dam']
data_fill1_columns =['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1','HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1','HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',
                          'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1','HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1','HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1',
                          'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1','HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1','HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1',
                          'HEAD Standby Position X Collect Result_Fill1','HEAD Standby Position Y Collect Result_Fill1','HEAD Standby Position Z Collect Result_Fill1','Head Clean Position X Collect Result_Fill1',
                          'Head Clean Position Y Collect Result_Fill1','Head Clean Position Z Collect Result_Fill1','Head Purge Position X Collect Result_Fill1','Head Purge Position Y Collect Result_Fill1',
                          'Head Purge Position Z Collect Result_Fill1','Machine Tact time Collect Result_Fill1','PalletID Collect Result_Fill1','Production Qty Collect Result_Fill1','Receip No Collect Result_Fill1',
                          'WorkMode Collect Result_Fill1']
data_fill2_columns = ['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2','HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2','HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2',
                         'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2','HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2','HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2',
                         'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2','HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2','HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2',
                         'HEAD Standby Position X Collect Result_Fill2','HEAD Standby Position Y Collect Result_Fill2','HEAD Standby Position Z Collect Result_Fill2','Head Clean Position X Collect Result_Fill2',
                         'Head Clean Position Y Collect Result_Fill2','Head Clean Position Z Collect Result_Fill2','Head Purge Position X Collect Result_Fill2','Head Purge Position Y Collect Result_Fill2',
                         'Head Purge Position Z Collect Result_Fill2','Machine Tact time Collect Result_Fill2','PalletID Collect Result_Fill2','Production Qty Collect Result_Fill2','Receip No Collect Result_Fill2',
                         'WorkMode Collect Result_Fill2']
data_dam = train_data[data_dam_columns]   
data_fill1 = train_data[data_fill1_columns]
data_fill2 = train_data[data_fill2_columns]

# OK 또는 NaN 값이 있는 행 식별
def check_and_shift_row(row):
    # 조건에 맞는지 확인
    if any((row == 'OK') | (row.isna())):
        # 열을 한 칸씩 앞으로 밀고 마지막 열을 NaN으로 설정
        row = row.shift(-1)
        row.iloc[-1] = np.nan
    return row

# 각 행에 대해 check_and_shift_row 함수를 적용
data_dam = data_dam.apply(check_and_shift_row, axis=1)
data_fill1 = data_fill1.apply(check_and_shift_row, axis=1)
data_fill2 = data_fill2.apply(check_and_shift_row, axis=1)

data_dam.drop('WorkMode Collect Result_Dam', axis=1, inplace=True)
data_fill1.drop('WorkMode Collect Result_Fill1', axis=1, inplace=True)
data_fill2.drop('WorkMode Collect Result_Fill2', axis=1, inplace=True)

train_data[data_dam.columns] = data_dam
train_data[data_fill1.columns] = data_fill1
train_data[data_fill2.columns] = data_fill2

## 2. 데이터 전처리

### 전처리 함수

In [4]:
#전처리 함수

# 파트 1: 전처리 초기 단계
def preprocess_initial(df):
    # 고유값이 하나인 컬럼 찾기
    unique_value_1_columns = [column for column in df.columns if df[column].nunique() == 1]

    # 모든 행의 값이 다른 컬럼 찾기
    row_count = len(df)
    matching_row_columns = [column for column in df.columns if df[column].value_counts().size == row_count]

    # 고유값 피쳐 제거
    df.drop(columns=unique_value_1_columns, inplace=True)
    df.drop(columns=matching_row_columns, inplace=True)
    return df

# 파트 2: 유사한 피쳐 제거
def reduce_dataframe(df):
    def get_value_counts_ratio(series):
        value_counts = series.value_counts(normalize=True)
        return value_counts.sort_values().values

    # 고유값 매핑 확인 함수
    def check_value_mapping(df, col1, col2):
        unique_values_1 = df[col1].unique()
        unique_values_2 = df[col2].unique()

        if len(unique_values_1) != len(unique_values_2):
            return False

        value_mapping = {}
        for val1 in unique_values_1:
            corresponding_values = df[df[col1] == val1][col2].unique()
            if len(corresponding_values) != 1:
                return False
            value_mapping[val1] = corresponding_values[0]

        for val1 in unique_values_1:
            ratio_1 = (df[col1] == val1).mean()
            ratio_2 = (df[col2] == value_mapping[val1]).mean()
            if ratio_1 != ratio_2:
                return False

        return True

    def compare_all_features(df):
        ratios = {column: get_value_counts_ratio(df[column]) for column in df.columns}
        similar_columns_dict = {column: [] for column in df.columns}

        # 고유값 비율이 같은 열들 찾기
        columns = list(ratios.keys())
        for i in range(len(columns)):
            for j in range(i + 1, len(columns)):
                if np.array_equal(ratios[columns[i]], ratios[columns[j]]):
                    similar_columns_dict[columns[i]].append(columns[j])

        # 고유값 비율이 같고 매핑도 동일한 피쳐들 찾기
        comparisons = []
        for key, values in similar_columns_dict.items():
            for value in values:
                if check_value_mapping(df, key, value):
                    comparisons.append((key, value))

        # if comparisons:
        #     print("다음 피쳐 쌍은 고유값 비율과 양상이 동일합니다:")
        #     for base, compare in comparisons:
        #         print(f"'{base}'와(과) '{compare}'")
        # else:
        #     print("모든 피쳐의 고유값 비율과 양상이 동일하지 않습니다.")

        return comparisons

    comparisons = compare_all_features(df)
    columns_to_remove = set()
    for _, col_to_remove in comparisons:
        columns_to_remove.add(col_to_remove)

    df = df.drop(columns=columns_to_remove)
    global df_columns
    df_columns = df.columns
    return df

# train 전처리 함수
def preprocess_train_dataframe(df):
    df = preprocess_initial(df)
    # df = reduce_dataframe(df)
    return df

# test 전처리 함수
def preprocess_test_dataframe(df):
    df = preprocess_initial(df)
    return df

In [5]:
df_train_pre = preprocess_train_dataframe(train_data)
features = df_train_pre.columns
features = features.drop('target')
df_order = df_train_pre.copy()

### 숫자형 문자 숫자형 변환 처리

In [6]:
def preprocess_coordinates(df, columns_to_replace):
    
    # 3. 모든 숫자형 문자열을 실제 숫자형으로 변환
    for column in columns_to_replace:
        if df[column].dtype == 'object':
            df[column] = pd.to_numeric(df[column], errors='coerce')
    
    return df

columns_to_replace = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]

In [7]:
# 결측치 함수 적용
df_concat = preprocess_coordinates(df_order, columns_to_replace)

### 오버 샘플링 후 target 비율 조정

In [8]:
# 원본 데이터에서 object 타입 열을 제외
df_numeric_only = df_concat.select_dtypes(include=['int64', 'float64'])
# 'target' 열 추가 (필요하다면 이 열의 타입도 확인)
df_numeric_only['target'] = df_concat['target']

In [9]:
normal_ratio = 1.0  # 1.0 means 1:1 ratio

df_normal = df_numeric_only[df_numeric_only["target"] == "Normal"]
df_abnormal = df_numeric_only[df_numeric_only["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(
    n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE
)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

  Total: Normal: 38156, AbNormal: 2350


target
AbNormal    2350
Normal      2350
dtype: int64

### 피쳐 생성 함수

In [10]:
def feature_engineering(df):

    # 1. 시간 관련 피처 생성
    df['Resin_Time_Diff_Stage1_2_Dam'] = df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam'] - df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
    df['Resin_Time_Diff_Stage1_3_Dam'] = df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam'] - df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
    df['Resin_Time_Diff_Stage2_3_Dam'] = df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam'] - df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']
    df['Resin_Time_Diff_Stage1_2_Fill1'] = df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1'] - df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
    df['Resin_Time_Diff_Stage1_3_Fill1'] = df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1'] - df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
    df['Resin_Time_Diff_Stage2_3_Fill1'] = df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1'] - df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']

    df['Machine_Tact_Time_Dam_Fill1_Diff'] = df['Machine Tact time Collect Result_Dam'] - df['Machine Tact time Collect Result_Fill1']
    df['Machine_Tact_Time_Fill1_Fill2_Diff'] = df['Machine Tact time Collect Result_Fill1'] - df['Machine Tact time Collect Result_Fill2']



    # 2. 속도 관련 피처 생성
    df['Stage1_Circle_Avg_Speed_Dam'] = (
        df['Stage1 Circle1 Distance Speed Collect Result_Dam'] +
        df['Stage1 Circle2 Distance Speed Collect Result_Dam'] +
        df['Stage1 Circle3 Distance Speed Collect Result_Dam'] +
        df['Stage1 Circle4 Distance Speed Collect Result_Dam']
    ) / 4
    
    df['Stage1_Line_Avg_Speed_Dam'] = (
        df['Stage1 Line1 Distance Speed Collect Result_Dam'] +
        df['Stage1 Line2 Distance Speed Collect Result_Dam'] +
        df['Stage1 Line3 Distance Speed Collect Result_Dam'] +
        df['Stage1 Line4 Distance Speed Collect Result_Dam']
    ) / 4
    
    df['Stage2_Circle_Avg_Speed_Dam'] = (
        df['Stage2 Circle1 Distance Speed Collect Result_Dam'] +
        df['Stage2 Circle2 Distance Speed Collect Result_Dam'] +
        df['Stage2 Circle3 Distance Speed Collect Result_Dam'] +
        df['Stage2 Circle4 Distance Speed Collect Result_Dam']
    ) / 4
    
    df['Stage2_Line_Avg_Speed_Dam'] = (
        df['Stage2 Line1 Distance Speed Collect Result_Dam'] +
        df['Stage2 Line2 Distance Speed Collect Result_Dam'] +
        df['Stage2 Line3 Distance Speed Collect Result_Dam'] +
        df['Stage2 Line4 Distance Speed Collect Result_Dam']
    ) / 4
    
    df['Stage3_Circle_Avg_Speed_Dam'] = (
        df['Stage3 Circle1 Distance Speed Collect Result_Dam'] +
        df['Stage3 Circle2 Distance Speed Collect Result_Dam'] +
        df['Stage3 Circle3 Distance Speed Collect Result_Dam'] +
        df['Stage3 Circle4 Distance Speed Collect Result_Dam']
    ) / 4
    
    df['Stage3_Line_Avg_Speed_Dam'] = (
        df['Stage3 Line1 Distance Speed Collect Result_Dam'] +
        df['Stage3 Line2 Distance Speed Collect Result_Dam'] +
        df['Stage3 Line3 Distance Speed Collect Result_Dam'] +
        df['Stage3 Line4 Distance Speed Collect Result_Dam']
    ) / 4

    # 각 스테이지의 원형 및 직선 속도의 차이 계산
    df['Stage1_Circle_Speed_Diff_Dam'] = df['Stage1 Circle2 Distance Speed Collect Result_Dam'] - df['Stage1 Circle1 Distance Speed Collect Result_Dam']
    df['Stage2_Circle_Speed_Diff_Dam'] = df['Stage2 Circle2 Distance Speed Collect Result_Dam'] - df['Stage2 Circle1 Distance Speed Collect Result_Dam']
    df['Stage3_Circle_Speed_Diff_Dam'] = df['Stage3 Circle2 Distance Speed Collect Result_Dam'] - df['Stage3 Circle1 Distance Speed Collect Result_Dam']

    df['Stage1_Line_Speed_Diff_Dam'] = df['Stage1 Line2 Distance Speed Collect Result_Dam'] - df['Stage1 Line1 Distance Speed Collect Result_Dam']
    df['Stage2_Line_Speed_Diff_Dam'] = df['Stage2 Line2 Distance Speed Collect Result_Dam'] - df['Stage2 Line1 Distance Speed Collect Result_Dam']
    df['Stage3_Line_Speed_Diff_Dam'] = df['Stage3 Line2 Distance Speed Collect Result_Dam'] - df['Stage3 Line1 Distance Speed Collect Result_Dam']

    # 각 스테이지의 총 속도 (원형 + 직선) 계산
    df['Total_Speed_Stage1_Dam'] = (
        df['Stage1 Circle1 Distance Speed Collect Result_Dam'] +
        df['Stage1 Circle2 Distance Speed Collect Result_Dam'] +
        df['Stage1 Circle3 Distance Speed Collect Result_Dam'] +
        df['Stage1 Circle4 Distance Speed Collect Result_Dam'] +
        df['Stage1 Line1 Distance Speed Collect Result_Dam'] +
        df['Stage1 Line2 Distance Speed Collect Result_Dam'] +
        df['Stage1 Line3 Distance Speed Collect Result_Dam'] +
        df['Stage1 Line4 Distance Speed Collect Result_Dam']
    )

    df['Total_Speed_Stage2_Dam'] = (
        df['Stage2 Circle1 Distance Speed Collect Result_Dam'] +
        df['Stage2 Circle2 Distance Speed Collect Result_Dam'] +
        df['Stage2 Circle3 Distance Speed Collect Result_Dam'] +
        df['Stage2 Circle4 Distance Speed Collect Result_Dam'] +
        df['Stage2 Line1 Distance Speed Collect Result_Dam'] +
        df['Stage2 Line2 Distance Speed Collect Result_Dam'] +
        df['Stage2 Line3 Distance Speed Collect Result_Dam'] +
        df['Stage2 Line4 Distance Speed Collect Result_Dam']
    )

    df['Total_Speed_Stage3_Dam'] = (
        df['Stage3 Circle1 Distance Speed Collect Result_Dam'] +
        df['Stage3 Circle2 Distance Speed Collect Result_Dam'] +
        df['Stage3 Circle3 Distance Speed Collect Result_Dam'] +
        df['Stage3 Circle4 Distance Speed Collect Result_Dam'] +
        df['Stage3 Line1 Distance Speed Collect Result_Dam'] +
        df['Stage3 Line2 Distance Speed Collect Result_Dam'] +
        df['Stage3 Line3 Distance Speed Collect Result_Dam'] +
        df['Stage3 Line4 Distance Speed Collect Result_Dam']
    )

    # 전체 총 속도 계산
    df['Total_Speed_Dam'] = (
        df['Total_Speed_Stage1_Dam'] +
        df['Total_Speed_Stage2_Dam'] +
        df['Total_Speed_Stage3_Dam']
    )


    # 3. 볼륨 관련 피처 생성
    # Dispense Volume 비율 계산
    df['Dispense_Volume_Ratio_Stage1_2_Dam'] = df['Dispense Volume(Stage1) Collect Result_Dam'] / df['Dispense Volume(Stage2) Collect Result_Dam']
    df['Dispense_Volume_Ratio_Stage2_3_Dam'] = df['Dispense Volume(Stage2) Collect Result_Dam'] / df['Dispense Volume(Stage3) Collect Result_Dam']
    df['Dispense_Volume_Ratio_Stage1_3_Dam'] = df['Dispense Volume(Stage1) Collect Result_Dam'] / df['Dispense Volume(Stage3) Collect Result_Dam']
    
    df['Dispense_Volume_Ratio_Stage1_2_Fill1'] = df['Dispense Volume(Stage1) Collect Result_Fill1'] / df['Dispense Volume(Stage2) Collect Result_Fill1']
    df['Dispense_Volume_Ratio_Stage2_3_Fill1'] = df['Dispense Volume(Stage2) Collect Result_Fill1'] / df['Dispense Volume(Stage3) Collect Result_Fill1']
    df['Dispense_Volume_Ratio_Stage1_3_Fill1'] = df['Dispense Volume(Stage1) Collect Result_Fill1'] / df['Dispense Volume(Stage3) Collect Result_Fill1']

    # Dispense Volume 차이 계산
    df['Dispense_Volume_Diff_Stage1_2_Dam'] = df['Dispense Volume(Stage1) Collect Result_Dam'] - df['Dispense Volume(Stage2) Collect Result_Dam']
    df['Dispense_Volume_Diff_Stage2_3_Dam'] = df['Dispense Volume(Stage2) Collect Result_Dam'] - df['Dispense Volume(Stage3) Collect Result_Dam']
    df['Dispense_Volume_Diff_Stage1_3_Dam'] = df['Dispense Volume(Stage1) Collect Result_Dam'] - df['Dispense Volume(Stage3) Collect Result_Dam']

    df['Dispense_Volume_Diff_Stage1_2_Fill1'] = df['Dispense Volume(Stage1) Collect Result_Fill1'] - df['Dispense Volume(Stage2) Collect Result_Fill1']
    df['Dispense_Volume_Diff_Stage2_3_Fill1'] = df['Dispense Volume(Stage2) Collect Result_Fill1'] - df['Dispense Volume(Stage3) Collect Result_Fill1']
    df['Dispense_Volume_Diff_Stage1_3_Fill1'] = df['Dispense Volume(Stage1) Collect Result_Fill1'] - df['Dispense Volume(Stage3) Collect Result_Fill1']

    # 각 스테이지의 Dispense Volume 총합 계산
    df['Total_Volume_Dam'] = (
        df['Dispense Volume(Stage1) Collect Result_Dam'] +
        df['Dispense Volume(Stage2) Collect Result_Dam'] +
        df['Dispense Volume(Stage3) Collect Result_Dam']
    )

    df['Total_Volume_Fill1'] = (
        df['Dispense Volume(Stage1) Collect Result_Fill1'] +
        df['Dispense Volume(Stage2) Collect Result_Fill1'] +
        df['Dispense Volume(Stage3) Collect Result_Fill1']
    )

    # 각 스테이지가 전체에서 차지하는 Dispense Volume 비율
    df['Stage1_to_Total_Volume_Ratio_Dam'] = df['Dispense Volume(Stage1) Collect Result_Dam'] / df['Total_Volume_Dam']
    df['Stage2_to_Total_Volume_Ratio_Dam'] = df['Dispense Volume(Stage2) Collect Result_Dam'] / df['Total_Volume_Dam']
    df['Stage3_to_Total_Volume_Ratio_Dam'] = df['Dispense Volume(Stage3) Collect Result_Dam'] / df['Total_Volume_Dam']

    df['Stage1_to_Total_Volume_Ratio_Fill1'] = df['Dispense Volume(Stage1) Collect Result_Fill1'] / df['Total_Volume_Fill1']
    df['Stage2_to_Total_Volume_Ratio_Fill1'] = df['Dispense Volume(Stage2) Collect Result_Fill1'] / df['Total_Volume_Fill1']
    df['Stage3_to_Total_Volume_Ratio_Fill1'] = df['Dispense Volume(Stage3) Collect Result_Fill1'] / df['Total_Volume_Fill1']



    # 4. 좌표 관련 피처 생성
    # X, Y, Z 축 좌표 차이 계산
    df['Head_X_Coord_Diff_Stage2_3_Dam'] = df['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam']
    df['Head_Y_Coord_Diff_Stage1_2_Dam'] = df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam']
    df['Head_Z_Coord_Diff_Stage1_2_Dam'] = df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam']

    df['Head_X_Coord_Diff_Stage2_3_Fill1'] = df['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1']
    df['Head_Y_Coord_Diff_Stage1_2_Fill1'] = df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1']
    df['Head_Z_Coord_Diff_Stage1_2_Fill1'] = df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1']

    df['Head_X_Coord_Diff_Stage2_3_Fill2'] = df['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'] - df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2']
    df['Head_Y_Coord_Diff_Stage1_2_Fill2'] = df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2']
    df['Head_Z_Coord_Diff_Stage1_2_Fill2'] = df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2']

    # 유클리드 거리 계산
    df['Head_Coord_Distance_Stage1_2_Dam'] = np.sqrt(
        (df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']) ** 2 +
        (df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam']) ** 2 +
        (df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam']) ** 2
    )
    df['Head_Coord_Distance_Stage2_3_Dam'] = np.sqrt(
        (df['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam']) ** 2 +
        (df['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam']) ** 2 +
        (df['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam']) ** 2
    )

    df['Head_Coord_Distance_Stage1_2_Fill1'] = np.sqrt(
        (df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']) ** 2 +
        (df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1']) ** 2 +
        (df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1']) ** 2
    )
    df['Head_Coord_Distance_Stage2_3_Fill1'] = np.sqrt(
        (df['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1']) ** 2 +
        (df['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1']) ** 2 +
        (df['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1']) ** 2
    )

    df['Head_Coord_Distance_Stage1_2_Fill2'] = np.sqrt(
        (df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2'] - df['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2']) ** 2 +
        (df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2']) ** 2 +
        (df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2']) ** 2
    )
    df['Head_Coord_Distance_Stage2_3_Fill2'] = np.sqrt(
        (df['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'] - df['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2']) ** 2 +
        (df['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2'] - df['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2']) ** 2 +
        (df['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'] - df['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2']) ** 2
    )



    # 6. 압력 관련 피처 생성
    # 압력 차이 계산
    df['Pressure_Diff_1st_2nd'] = df['1st Pressure Collect Result_AutoClave'] - df['2nd Pressure Collect Result_AutoClave']
    df['Pressure_Diff_2nd_3rd'] = df['2nd Pressure Collect Result_AutoClave'] - df['3rd Pressure Collect Result_AutoClave']
    df['Pressure_Diff_1st_3rd'] = df['1st Pressure Collect Result_AutoClave'] - df['3rd Pressure Collect Result_AutoClave']

    # 압력 비율 계산
    df['Pressure_Ratio_1st_2nd'] = df['1st Pressure Collect Result_AutoClave'] / df['2nd Pressure Collect Result_AutoClave']
    df['Pressure_Ratio_2nd_3rd'] = df['2nd Pressure Collect Result_AutoClave'] / df['3rd Pressure Collect Result_AutoClave']
    df['Pressure_Ratio_1st_3rd'] = df['1st Pressure Collect Result_AutoClave'] / df['3rd Pressure Collect Result_AutoClave']

    # 압력 평균값 계산
    df['Pressure_Avg'] = (
        df['1st Pressure Collect Result_AutoClave'] +
        df['2nd Pressure Collect Result_AutoClave'] +
        df['3rd Pressure Collect Result_AutoClave']
    ) / 3


    # 7. 두께 관련 피처 생성
    # 두께 차이 계산
    df['Thickness_Diff_1_2'] = df['THICKNESS 1 Collect Result_Dam'] - df['THICKNESS 2 Collect Result_Dam']
    df['Thickness_Diff_2_3'] = df['THICKNESS 2 Collect Result_Dam'] - df['THICKNESS 3 Collect Result_Dam']
    df['Thickness_Diff_1_3'] = df['THICKNESS 1 Collect Result_Dam'] - df['THICKNESS 3 Collect Result_Dam']

    # 두께 평균값 계산
    df['Thickness_Avg'] = (
        df['THICKNESS 1 Collect Result_Dam'] +
        df['THICKNESS 2 Collect Result_Dam'] +
        df['THICKNESS 3 Collect Result_Dam']
    ) / 3

    # 두께 표준편차 계산
    df['Thickness_Std'] = df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam', 'THICKNESS 3 Collect Result_Dam']].std(axis=1)



    return df

In [11]:
# 피쳐 생성 함수 적용
df_concat = feature_engineering(df_concat)

# Scaler
numeric_columns = df_concat.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
df_concat[numeric_columns] = scaler.fit_transform(df_concat[numeric_columns])

train_x = df_concat.copy()

In [12]:
y = train_x.target
train_x.drop(columns = ['target'],inplace =True)
train_x = train_x.select_dtypes(exclude=['object'])

### PCA

In [13]:
# # PCA 모델 생성
# pca = PCA(n_components=0.95)  # 설명된 분산이 95%가 될 때까지 성분 수를 결정
# train_x_pca = pca.fit_transform(train_x)

In [14]:
# print("원래 피처 수:", train_x.shape[1])
# print("PCA 후 성분 수:", train_x_pca.shape[1])
# print("각 주성분의 설명된 분산 비율:", pca.explained_variance_ratio_)
# print("전체 설명된 분산 비율:", pca.explained_variance_ratio_.sum())

In [15]:
# # 주성분 이름을 생성 (PC1, PC2, ...)
# pca_columns = [f'PC{i+1}' for i in range(train_x_pca.shape[1])]

# # PCA 데이터프레임 생성
# df_pca = pd.DataFrame(train_x_pca, columns=pca_columns)

# # # 타겟 변수 추가
# # df_pca['target'] = y

### SHAP

In [16]:
# 타겟 변수 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [17]:
# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(train_x, y_encoded, test_size=0.2, random_state=42)

# XGBoost 분류기 모델 초기화
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# F1 스코어 계산
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

# SHAP 값 계산
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# SHAP 값의 평균 절대값 계산
shap_values_df = pd.DataFrame(shap_values, columns=X_test.columns)
shap_importance = shap_values_df.abs().mean().sort_values(ascending=False)
shap_importance

F1 Score: 0.5586


[18:05:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


Production Qty Collect Result_Dam                             0.315817
Production Qty Collect Result_Fill2                           0.306603
Machine Tact time Collect Result_Dam                          0.153716
Head_Coord_Distance_Stage1_2_Fill1                            0.150014
Machine Tact time Collect Result_Fill2                        0.144858
Pressure_Ratio_1st_3rd                                        0.124339
Machine_Tact_Time_Dam_Fill1_Diff                              0.113078
Pressure_Avg                                                  0.109280
WorkMode Collect Result_Fill1                                 0.107700
Pressure_Ratio_2nd_3rd                                        0.106133
Machine Tact time Collect Result_Fill1                        0.104883
PalletID Collect Result_Dam                                   0.099780
Chamber Temp. Collect Result_AutoClave                        0.098494
Machine_Tact_Time_Fill1_Fill2_Diff                            0.095517
Pallet

In [18]:
# shap 피쳐 기반
important_features = shap_importance[shap_importance > 0].index.tolist()
X_train_shap = X_train[important_features]
X_test_shap = X_test[important_features]

model.fit(X_train_shap, y_train)
y_pred = model.predict(X_test_shap)

# F1 스코어 계산
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

F1 Score: 0.5617


In [19]:
# SHAP 값이 0.01 이상인 피처만 선택
important_features = shap_importance[shap_importance > 0].index.tolist()
train_x_shap = train_x[important_features]

## 3. 모델

In [20]:
catboost_model = CatBoostClassifier(verbose=0, random_state=42)
xgboost_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
randomforest_model = RandomForestClassifier(random_state=42)

# 앙상블 모델 정의 (VotingClassifier)
voting_clf = VotingClassifier(
    estimators=[
        ('catboost', catboost_model),
        ('xgboost', xgboost_model),
        ('randomforest', randomforest_model)
    ],
    voting='soft',
     weights=[4, 4, 3]
)

# 앙상블 모델 학습
voting_clf.fit(train_x_shap, y)

VotingClassifier(estimators=[('catboost',
                              <catboost.core.CatBoostClassifier object at 0x000001F1959B2810>),
                             ('xgboost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None, gamma=...
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('randomforest',
                              RandomForestClassifier(random_state=42))],
                 voting='soft', weights=[4, 4, 3])

## 4. 제출하기

### 테스트 데이터 

In [31]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

In [32]:
test_data.head()

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Z Unit Time_Dam,CURE END POSITION Z Judge Value_Dam,CURE END POSITION Θ Collect Result_Dam,CURE END POSITION Θ Unit Time_Dam,CURE END POSITION Θ Judge Value_Dam,CURE SPEED Collect Result_Dam,CURE SPEED Unit Time_Dam,CURE SPEED Judge Value_Dam,CURE STANDBY POSITION X Collect Result_Dam,CURE STANDBY POSITION X Unit Time_Dam,CURE STANDBY POSITION X Judge Value_Dam,CURE STANDBY POSITION Z Collect Result_Dam,CURE STANDBY POSITION Z Unit Time_Dam,CURE STANDBY POSITION Z Judge Value_Dam,CURE STANDBY POSITION Θ Collect Result_Dam,CURE STANDBY POSITION Θ Unit Time_Dam,CURE STANDBY POSITION Θ Judge Value_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION X Unit Time_Dam,CURE START POSITION X Judge Value_Dam,CURE START POSITION Z Collect Result_Dam,CURE START POSITION Z Unit Time_Dam,CURE START POSITION Z Judge Value_Dam,CURE START POSITION Θ Collect Result_Dam,CURE START POSITION Θ Unit Time_Dam,CURE START POSITION Θ Judge Value_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED SPEED OF RESIN Unit Time_Dam,DISCHARGED SPEED OF RESIN Judge Value_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Unit Time_Dam,DISCHARGED TIME OF RESIN(Stage1) Judge Value_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Unit Time_Dam,DISCHARGED TIME OF RESIN(Stage2) Judge Value_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Unit Time_Dam,DISCHARGED TIME OF RESIN(Stage3) Judge Value_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage1) Unit Time_Dam,Dispense Volume(Stage1) Judge Value_Dam,Dispense Volume(Stage2) Collect Result_Dam,Dispense Volume(Stage2) Unit Time_Dam,Dispense Volume(Stage2) Judge Value_Dam,Dispense Volume(Stage3) Collect Result_Dam,Dispense Volume(Stage3) Unit Time_Dam,Dispense Volume(Stage3) Judge Value_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Unit Time_Dam,HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Unit Time_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Judge Value_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Unit Time_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Judge Value_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Unit Time_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage1) Judge Value_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Unit Time_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage2) Judge Value_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Unit Time_Dam,HEAD NORMAL COORDINATE Y AXIS(Stage3) Judge Value_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Unit Time_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Judge Value_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Unit Time_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage2) Judge Value_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Unit Time_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage3) Judge Value_Dam,HEAD Standby Position X Collect Result_Dam,HEAD Standby Position X Unit Time_Dam,HEAD Standby Position X Judge Value_Dam,HEAD Standby Position Y Collect Result_Dam,HEAD Standby Position Y Unit Time_Dam,HEAD Standby Position Y Judge Value_Dam,HEAD Standby Position Z Collect Result_Dam,HEAD Standby Position Z Unit Time_Dam,HEAD Standby Position Z Judge Value_D

In [21]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

drop_cols = []
for column in test_data.columns:
    if (test_data[column].notnull().sum() // 2) < test_data[column].isnull().sum():
        drop_cols.append(column)
test_data = test_data.drop(drop_cols, axis=1)

In [22]:
data_dam = test_data[data_dam_columns]   
data_fill1 = test_data[data_fill1_columns]
data_fill2 = test_data[data_fill2_columns]

data_dam = data_dam.apply(check_and_shift_row, axis=1)
data_fill1 = data_fill1.apply(check_and_shift_row, axis=1)
data_fill2 = data_fill2.apply(check_and_shift_row, axis=1)

data_dam.drop('WorkMode Collect Result_Dam', axis=1, inplace=True)
data_fill1.drop('WorkMode Collect Result_Fill1', axis=1, inplace=True)
data_fill2.drop('WorkMode Collect Result_Fill2', axis=1, inplace=True)

test_data[data_dam.columns] = data_dam
test_data[data_fill1.columns] = data_fill1
test_data[data_fill2.columns] = data_fill2

In [23]:
df_test = preprocess_test_dataframe(test_data)

df_test_pre = df_test.copy()
df_test_pre = df_test_pre[features]

In [24]:
#결측값 함수
df_test_pre = preprocess_coordinates(df_test_pre, columns_to_replace)

In [25]:
# 피쳐 생성 함수
df_test_pre = feature_engineering(df_test_pre)
numeric_columns = df_test_pre.select_dtypes(include=['float64', 'int64']).columns

# 스케일러
scaler = StandardScaler()
df_test_pre[numeric_columns] = scaler.fit_transform(df_test_pre[numeric_columns])

# SHAP 피쳐 test 셋 적용
test_x_shap = df_test_pre[important_features]

In [26]:
y_pred = voting_clf.predict(test_x_shap)

### 제출 파일 작성

In [27]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = y_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**